<p style="background-color:powderblue; font-size:125%; color: rebeccapurple;">Выполните задания</p>
<p>Выполните задания с помощью соедиинения таблиц в Jupyter Notebook </p>

1. Создайте объединение двух запросов, которые выдают значения полей *univ_name*, *city*, *rating* для всех университетов. Те из них, у которых рейтинг равен или выше 300, должны иметь комментарий ‘высокий’, все остальные — ‘низкий’.

2. Напишите команду, которая выдает список фамилий студентов с комментарием: 'успевает' у студентов, имеющих все положительные оценки; 'не успевает' для сдававших экзамены, но имеющих хотя бы одну неудовлетворительную оценку; 'не сдавал' — для всех остальных. В выводимом результате фамилии студентов упорядочите по алфавиту.

3. Выведите объединенный список студентов и преподавателей, живущих в Москве, с соответствующими комментариями 'студент' или 'преподаватель'.

4. Выведите объединенный список студентов и преподавателей ВГУ с соответствующими комментариями 'студент' или 'преподаватель'.

5. Для каждого города выведите названия университетов с минимальным и максимальным для данного города рейтингом. Пометьте строки списка словами ‘min’ и ‘max’, поместив их в дополнительном столбце. Резулььтат должен содержать название города, название университета, рейтинг. Упорядочите результат по городу.

6. Для каждого курса выведите фамилии студентов, получающих минимальные и максимальные на их курсе стипендии. Пометьте строки списка словами ‘min’ и ‘max’, поместив их в дополнительном столбце.

7. Для каждого курса выведите фамилии самого старшего и самого младшего студентов. Пометьте строки списка словами 'младший' и 'старший', поместив их в дополнительном столбце.

8. Напишите запрос для получения полного списка университетов вместе с количеством студентов, которые в них учатся. Для университетов, не имеющих студентов, поместите в список фразу ‘Студентов нет’. отсортируйте результат по убыванию числа студентов.

9. Напишите запрос для получения полного списка университетов вместе с фамилиями преподавателей, в них работающих. Для университетов, не имеющих преподавателей, поместите в список фразу ‘Преподавателей нет’. результат должен содержать название университета, фамилию и имя преподавателя. отсортируйте результат по фамилии и имени преподавателя.

10. Выведите полный список студентов вместе с оценками, полученными ими на экзаменах. Для студентов, не сдававших экзамены, в поле оценки поместите 0.

In [3]:
from sqlalchemy import MetaData, Table, create_engine, select, and_, or_, not_, text, desc
from sqlalchemy import union, union_all
from sqlalchemy.sql import func, distinct, exists
import pandas as pd

In [4]:
engine = create_engine('sqlite:///../Data/Students.db')
metadata_obj = MetaData() # Объект-контейнер, содержит информацию о схеме данных
metadata_obj.reflect(bind=engine) # Отображение БД

Создание объектов таблиц

In [5]:
exam_marks=Table("exam_marks", metadata_obj, autoload=True, autoload_with="engine")
lecturer=Table("lecturer", metadata_obj, autoload=True, autoload_with="engine")
student=Table("student", metadata_obj, autoload=True, autoload_with="engine")
subj_lect=Table("subj_lect", metadata_obj, autoload=True, autoload_with="engine")
subject=Table("subject", metadata_obj, autoload=True, autoload_with="engine")
university=Table("university", metadata_obj, autolad=True, autolad_with="engine")

In [6]:
conn=engine.connect() # Соединение

# Решение

In [7]:
st = student.alias('st')
em = exam_marks.alias('em')
u = university.alias('u')
l = lecturer.alias('l')
sl = subj_lect.alias('sl')
su = subject.alias('su')

In [11]:
# 1. Создайте объединение двух запросов, которые выдают значения полей *univ_name*, *city*, *rating* для всех университетов.
# Те из них, у которых рейтинг равен или выше 300, должны иметь комментарий ‘высокий’, все остальные — ‘низкий’.

s1 = select(text("'Высокий' AS rating"), u.c.univ_name, u.c.city, u.c.rating.label('Rating'))
s1 = s1.where(u.c.rating >= 300)

s2 = select(text("'Низкий' AS rating"), u.c.univ_name, u.c.city, u.c.rating)
s2 = s2.where(or_(u.c.rating < 300, u.c.rating == None))

us = union_all(s1, s2)

us = us.order_by(text("rating"), u.c.univ_name)

print(us)

pd.read_sql(us, conn)

SELECT 'Высокий' AS rating, u.univ_name, u.city, u.rating AS "Rating" 
FROM university AS u 
WHERE u.rating >= :rating_1 UNION ALL SELECT 'Низкий' AS rating, u.univ_name, u.city, u.rating 
FROM university AS u 
WHERE u.rating < :rating_2 OR u.rating IS NULL ORDER BY rating, univ_name


,rating,univ_name,city,Rating
0,Высокий,БГУ,Белгород,326.0
1,Высокий,ВГМА,Воронеж,327.0
2,Высокий,МГТУ им. Баумана,Москва,372.0
3,Высокий,МГУ,Москва,606.0
4,Высокий,МФТИ,Москва,500.0
5,Высокий,НГУ,Новосибирск,345.0
6,Высокий,РГУ,Ростов,416.0
7,Высокий,ТГУ,Томск,368.0
8,Высокий,"ФГБОУ ""Политех""",Санкт-Петербург,304.0
9,Высокий,ФинУ,Москва,330.0


In [11]:
# 2. Напишите команду, которая выдает список фамилий студентов с комментарием: 'успевает' у студентов,
# имеющих все положительные оценки; 'не успевает' для сдававших экзамены, но имеющих хотя бы одну неудовлетворительную оценку;
# 'не сдавал' — для всех остальных. В выводимом результате фамилии студентов упорядочите по алфавиту.

s1 = select(text("'Не успевает' AS успеваемость"), student)
s1 = s1.where(student.c.student_id == exam_marks.c.student_id, exam_marks.c.mark == None)

s2 = select(text("'Не успевает' AS успеваемость"), student)
s2 = s2.where(student.c.student_id == exam_marks.c.student_id, exam_marks.c.mark.in_([1,2,3])).distinct()

s3sq = select(st.c.student_id)
s3sq = s3sq.where(st.c.student_id == em.c.student_id, or_(em.c.mark.in_([1, 2, 3]), em.c.mark == None))
s3 = select(text("'Успевает' AS успеваемость"), student)
s3 = s3.where(student.c.student_id == exam_marks.c.student_id, exam_marks.c.mark.in_([4, 5]))
s3 = s3.where(student.c.student_id.not_in(s3sq))

s4sq = select(exam_marks.c.student_id)
s4 = select(text("'Не сдавал' AS успеваемость"), student)
s4 =s4.where(student.c.student_id.not_in(s4sq))

su = union_all(s1, s2, s3, s4).order_by(student.c.surname)

print(su)
pd.read_sql(su, conn)

SELECT 'Не успевает' AS успеваемость, student.student_id, student.surname, student.name, student.stipend, student.kurs, student.city, student.birthday, student.univ_id 
FROM student, exam_marks 
WHERE student.student_id = exam_marks.student_id AND exam_marks.mark IS NULL UNION ALL SELECT DISTINCT 'Не успевает' AS успеваемость, student.student_id, student.surname, student.name, student.stipend, student.kurs, student.city, student.birthday, student.univ_id 
FROM student, exam_marks 
WHERE student.student_id = exam_marks.student_id AND exam_marks.mark IN ([POSTCOMPILE_mark_1]) UNION ALL SELECT 'Успевает' AS успеваемость, student.student_id, student.surname, student.name, student.stipend, student.kurs, student.city, student.birthday, student.univ_id 
FROM student, exam_marks 
WHERE student.student_id = exam_marks.student_id AND exam_marks.mark IN ([POSTCOMPILE_mark_2]) AND (student.student_id NOT IN (SELECT st.student_id 
FROM student AS st, exam_marks AS em 
WHERE st.student_id = em.stude

,успеваемость,student_id,surname,name,stipend,kurs,city,birthday,univ_id
0,Не сдавал,700,123,312,123313,1,aaa,NaT,NaN
1,Не сдавал,78,Агафонов,Василий,150,1,Грозный,1994-10-29,10.0
2,Не сдавал,113,Аксенов,Валерий,150,1,Мурманск,1994-10-27,10.0
3,Не успевает,105,Антонов,Анатолий,100,1,Сыктывкар,1997-04-08,22.0
4,Не сдавал,201,Архипов,Андрей,180,1,Орел,2000-01-28,22.0
...,...,...,...,...,...,...,...,...,...
86,Не успевает,116,Федоров,Гостомысл,100,4,Новый Уренгой,2000-11-16,10.0
87,Не сдавал,80,Фролова,Василиса,100,3,Кисловодск,1999-06-02,44.0
88,Не сдавал,140,Ширяев,Андрей,150,2,Джубга,1997-09-13,46.0
89,Не сдавал,69,Шубина,Елена,0,2,Камень-на-Оби,2002-08-24,45.0


In [16]:
# 3. Выведите объединенный список студентов и преподавателей,
# живущих в Москве, с соответствующими комментариями 'студент' или 'преподаватель'.

s1 = select(text("'Студент' AS Статус"), student.c.name)
s1= s1.where(student.c.city == "Москва")

s2 = select(text("'Преподаватель' AS Статус"), lecturer.c.name)
s2 = s2.where(lecturer.c.city == "Москва")

su = union_all(s1, s2)

print(su)
pd.read_sql(su, conn)

SELECT 'Студент' AS Статус, student.name 
FROM student 
WHERE student.city = :city_1 UNION ALL SELECT 'Преподаватель' AS Статус, lecturer.name 
FROM lecturer 
WHERE lecturer.city = :city_2


,Статус,name
0,Студент,Вадим
1,Студент,Александр
2,Студент,Федор
3,Преподаватель,Павел
4,Преподаватель,Николай


In [17]:
# 4. Выведите объединенный список студентов и преподавателей ВГУ с соответствующими комментариями 'студент' или 'преподаватель'.
s1 = select(text("'Студент' AS Статус"), student.c.name)
s1 = s1.where(student.c.univ_id == university.c.univ_id, university.c.univ_name == "ВГУ")

s2 = select(text("'Преподаватель' AS Статус"), lecturer.c.name)
s2 = s2.where(lecturer.c.univ_id == university.c.univ_id, university.c.univ_name == "ВГУ")

su = union_all(s1, s2)

print(su)
pd.read_sql(su, conn)

SELECT 'Студент' AS Статус, student.name 
FROM student, university 
WHERE student.univ_id = university.univ_id AND university.univ_name = :univ_name_1 UNION ALL SELECT 'Преподаватель' AS Статус, lecturer.name 
FROM lecturer, university 
WHERE lecturer.univ_id = university.univ_id AND university.univ_name = :univ_name_2


,Статус,name
0,Студент,Иван
1,Студент,Петр
2,Студент,Борис
3,Студент,Ольга
4,Студент,Вадим
5,Студент,Вероника
6,Студент,Фёкла
7,Студент,Вероника
8,Студент,Дарья
9,Студент,Василий


In [18]:
#5. Для каждого города выведите названия университетов с минимальным и максимальным для данного города рейтингом.
# Пометьте строки списка словами ‘min’ и ‘max’, поместив их в дополнительном столбце.
# Резулььтат должен содержать название города, название университета, рейтинг. Упорядочите результат по городу.

s1 = select(text("'min' AS Rating"), university.c.city, university.c.univ_name, university.c.rating)
s1 =s1.group_by(university.c.city)
s1= s1.having(func.min(university.c.rating))

s2 = select(text("'max' AS Rating"), university.c.city, university.c.univ_name, university.c.rating)
s2= s2.group_by(university.c.city)
s2= s2.having(func.max(university.c.rating))

s3 = select(text("'min' AS Rating"), university.c.city, university.c.univ_name, university.c.rating)
s3 = s3.where(university.c.rating == None)
s3 = s3.group_by(university.c.city)
s4 = select(text("'max' AS Rating"), university.c.city, university.c.univ_name, university.c.rating)
s4 = s4.where(university.c.rating == None)
s4 = s4.group_by(university.c.city)

su = union_all(s1, s2, s3, s4).order_by(university.c.city)

print(su)
pd.read_sql(su, conn)

SELECT 'min' AS Rating, university.city, university.univ_name, university.rating 
FROM university GROUP BY university.city 
HAVING min(university.rating) UNION ALL SELECT 'max' AS Rating, university.city, university.univ_name, university.rating 
FROM university GROUP BY university.city 
HAVING max(university.rating) UNION ALL SELECT 'min' AS Rating, university.city, university.univ_name, university.rating 
FROM university 
WHERE university.rating IS NULL GROUP BY university.city UNION ALL SELECT 'max' AS Rating, university.city, university.univ_name, university.rating 
FROM university 
WHERE university.rating IS NULL GROUP BY university.city ORDER BY city


,Rating,city,univ_name,rating
0,min,Белгород,БГУ,326.0
1,max,Белгород,БГУ,326.0
2,min,Воронеж,ВГУ,296.0
3,max,Воронеж,ВГМА,327.0
4,min,Казань,КФУ,NaN
5,max,Казань,КФУ,NaN
6,min,Москва,ФинУ,330.0
7,max,Москва,МГУ,606.0
8,min,Новосибирск,НГУ,345.0
9,max,Новосибирск,НГУ,345.0


In [19]:
# 6. Для каждого курса выведите фамилии студентов, получающих минимальные и максимальные на их курсе стипендии.
# Пометьте строки списка словами ‘min’ и ‘max’, поместив их в дополнительном столбце.

s1 = select(text("'min' AS Stip"), student)
s1 = s1.group_by(student.c.kurs)
s1 = s1.having(or_(func.min(student.c.stipend), func.min(student.c.stipend) == 0))

s2 = select(text("'max' AS Stip"), student)
s2 = s2.group_by(student.c.kurs)
s2 = s2.having(func.max(student.c.stipend))

su = union_all(s1, s2)

print(su)
pd.read_sql(su, conn)

SELECT 'min' AS Stip, student.student_id, student.surname, student.name, student.stipend, student.kurs, student.city, student.birthday, student.univ_id 
FROM student GROUP BY student.kurs 
HAVING min(student.stipend) OR min(student.stipend) = :min_1 UNION ALL SELECT 'max' AS Stip, student.student_id, student.surname, student.name, student.stipend, student.kurs, student.city, student.birthday, student.univ_id 
FROM student GROUP BY student.kurs 
HAVING max(student.stipend)


,Stip,student_id,surname,name,stipend,kurs,city,birthday,univ_id
0,min,94,Васильева,Оксана,0,1,Кизилюрт,1994-05-14,32
1,min,10,Кузнецов,Борис,0,2,Брянск,2001-12-08,10
2,min,60,Рожкова,Пелагея,0,3,Рубцовск,2001-09-15,48
3,min,58,Журавлева,Вера,0,4,Новосибирск,2003-06-14,11
4,min,57,Кудряшова,Вера,100,5,Нижневартовск,2002-02-18,14
5,max,64,Жук,Оксана,250,1,Воркута,2000-11-16,47
6,max,12,Зайцева,Ольга,250,2,Липецк,2001-05-01,10
7,max,91,Блохина,Ираида,250,3,Новый Уренгой,1997-03-23,22
8,max,68,Казакова,Ольга,250,4,Кемерово,2000-03-29,44
9,max,55,Белкин,Вадим,250,5,Воронеж,2000-01-07,10


In [20]:
# 7. Для каждого курса выведите фамилии самого старшего и самого младшего студентов.
# Пометьте строки списка словами 'младший' и 'старший', поместив их в дополнительном столбце.

s1 = select(text("'Младший'"), student)
s1 = s1.group_by(student.c.kurs)
s1 = s1.having(func.min(func.date(student.c.birthday)))

s2 = select(text("'Старший'"), student)
s2 = s2.group_by(student.c.kurs)
s2 = s2.having(func.max(func.date(student.c.birthday)))

su = union_all(s1, s2)

print(su)
pd.read_sql(su, conn)

SELECT 'Младший', student.student_id, student.surname, student.name, student.stipend, student.kurs, student.city, student.birthday, student.univ_id 
FROM student GROUP BY student.kurs 
HAVING min(date(student.birthday)) UNION ALL SELECT 'Старший', student.student_id, student.surname, student.name, student.stipend, student.kurs, student.city, student.birthday, student.univ_id 
FROM student GROUP BY student.kurs 
HAVING max(date(student.birthday))


,'Младший',student_id,surname,name,stipend,kurs,city,birthday,univ_id
0,Младший,94,Васильева,Оксана,0,1,Кизилюрт,1994-05-14,32
1,Младший,131,Ковалев,Ефим,200,2,Волгоград,1995-11-28,47
2,Младший,83,Сорокина,Ирина,200,3,Усмань,1993-09-28,48
3,Младший,85,Казакова,Мария,150,4,Нальчик,1994-02-18,32
4,Младший,103,Трофимов,Владислав,250,5,Верхоянск,1995-03-24,11
5,Старший,15,Ордуханян,Рафаэль,120,1,Липецк,2003-06-01,18
6,Старший,66,Некрасова,Вероника,250,2,Иркутск,2004-08-23,10
7,Старший,62,Семенова,Вероника,0,3,Вологда,2003-10-04,10
8,Старший,58,Журавлева,Вера,0,4,Новосибирск,2003-06-14,11
9,Старший,144,Ситников,Викентий,150,5,Иркутск,2003-10-05,46


In [21]:
# 8. Напишите запрос для получения полного списка университетов вместе с количеством студентов, которые в них учатся.
# Для университетов, не имеющих студентов, поместите в список фразу ‘Студентов нет’.
# отсортируйте результат по убыванию числа студентов.

s1 = select(university, func.count(student.c.univ_id).label("Кол-во студентов"))
s1 = s1.join(student)
s1 = s1.group_by(university.c.univ_id)

s2sq = select(student.c.univ_id)
s2 = select(university, text("'Студентов нет'"))
s2 = s2.group_by(university.c.univ_id)
s2 = s2.where(university.c.univ_id.not_in(s2sq))

su = union_all(s1, s2)
print(su)
pd.read_sql(su, conn)

SELECT university.univ_id, university.univ_name, university.rating, university.city, count(student.univ_id) AS "Кол-во студентов" 
FROM university JOIN student ON university.univ_id = student.univ_id GROUP BY university.univ_id UNION ALL SELECT university.univ_id, university.univ_name, university.rating, university.city, 'Студентов нет' 
FROM university 
WHERE (university.univ_id NOT IN (SELECT student.univ_id 
FROM student)) GROUP BY university.univ_id


,univ_id,univ_name,rating,city,Кол-во студентов
0,10,ВГУ,296.0,Воронеж,16
1,11,НГУ,345.0,Новосибирск,4
2,14,БГУ,326.0,Белгород,9
3,15,ТГУ,368.0,Томск,2
4,18,ВГМА,327.0,Воронеж,7
5,22,МГУ,606.0,Москва,10
6,32,РГУ,416.0,Ростов,7
7,44,ФинУ,330.0,Москва,10
8,45,МГТУ им. Баумана,372.0,Москва,5
9,46,"ФГБОУ ""Политех""",304.0,Санкт-Петербург,9


In [22]:
# 9. Напишите запрос для получения полного списка университетов вместе с фамилиями преподавателей, в них работающих.
# Для университетов, не имеющих преподавателей, поместите в список фразу ‘Преподавателей нет’.
# результат должен содержать название университета, фамилию и имя преподавателя.
# отсортируйте результат по фамилии и имени преподавателя.

s1 = select(university.c.univ_name, lecturer.c.surname)
s1 = s1.where(university.c.univ_id == lecturer.c.univ_id)

s2sq = select(lecturer.c.univ_id)
s2 = select(university.c.univ_name, text("'Преподавателей нет'"))
s2 = s2.where(university.c.univ_id.not_in(s2sq))

su = union_all(s1, s2)

print(su)
pd.read_sql(su, conn)

SELECT university.univ_name, lecturer.surname 
FROM university, lecturer 
WHERE university.univ_id = lecturer.univ_id UNION ALL SELECT university.univ_name, 'Преподавателей нет' 
FROM university 
WHERE (university.univ_id NOT IN (SELECT lecturer.univ_id 
FROM lecturer))


,univ_name,surname
0,"ФГБОУ ""Политех""",Сафонова
1,ВГУ,Зайцева
2,НГУ,Мамонтова
3,ВГМА,Богданова
4,УЛГУ,Брагина
5,ВГУ,Максимова
6,УЛГУ,Дмитриева
7,ВГМА,Ситникова
8,ВГМА,Крюкова
9,ВГМА,Артемьева


In [12]:
# 10. Выведите полный список студентов вместе с оценками, полученными ими на экзаменах.
# Для студентов, не сдававших экзамены, в поле оценки поместите 0.

s1 = select(student, exam_marks.c.mark)
s1 = s1.where(student.c.student_id == exam_marks.c.student_id)

s2sq = select(exam_marks.c.student_id)
s2 = select(student, text("'0'"))
s2 = s2.where(student.c.student_id.not_in(s2sq))

su = union_all(s1, s2)

print(su)
pd.read_sql(su, conn)

SELECT student.student_id, student.surname, student.name, student.stipend, student.kurs, student.city, student.birthday, student.univ_id, exam_marks.mark 
FROM student, exam_marks 
WHERE student.student_id = exam_marks.student_id UNION ALL SELECT student.student_id, student.surname, student.name, student.stipend, student.kurs, student.city, student.birthday, student.univ_id, '0' 
FROM student 
WHERE (student.student_id NOT IN (SELECT exam_marks.student_id 
FROM exam_marks))


,student_id,surname,name,stipend,kurs,city,birthday,univ_id,mark
0,97,Вишнякова,Анжела,200,2,Иваново,2002-02-10,46.0,1
1,85,Казакова,Мария,150,4,Нальчик,1994-02-18,32.0,4
2,64,Жук,Оксана,250,1,Воркута,2000-11-16,47.0,4
3,126,Жуков,Пров,0,3,Кыштым,1998-09-11,11.0,1
4,71,Кондрат,Синицин,200,4,Ишим,2001-02-23,18.0,3
...,...,...,...,...,...,...,...,...,...
96,206,Киселев,Владимир,210,2,Орел,2003-02-10,44.0,0
97,207,Миронова,Федор,200,3,Москва,2002-04-15,45.0,0
98,265,Павлов,Андрей,0,3,Воронеж,2000-11-05,10.0,0
99,276,Петров,Антон,200,4,Ульяновск,2001-08-05,22.0,0
